In [1]:
import pandas as pd
import numpy as np

In [6]:
text=pd.read_csv("coupons_ner.csv",header=None)

In [7]:
text

,0
0,Save $2.00 ONE Downy Liquid Fabric Conditioner...
1,Save $2.00 ONE Tide PODS OR Tide Power PODS (e...
2,Save $2.00 ONE Tide Laundry Detergent (exclude...
3,SAVE $1.00 ON TWO when you buy TWO BOXES (8.9 ...
4,$3.00 OFF when you purchase any THREE (3) Pepp...
...,...
881,SAVE $2.00 when you buy $7 worth of participat...
882,SAVE $5.00 on any Gain scented purchase of $30...
883,SAVE $1.00 when you buy any ONE (1) NEW AXE pr...
884,BUY TWO GET ONE Buy ONE (1) CHIPS AHOY! Cookie...


### Exercise 1 - Returning the FaceValue as Output

In [8]:
import re

In [11]:
text[0][0]

'Save $2.00 ONE Downy Liquid Fabric Conditioner 72 ld or larger (includes Downy Odor Protect 48 oz or larger OR Downy Wrinkle Guard 40 oz or larger OR Downy Nature Blends 67 oz or larger) OR Bounce/Downy Sheets 130 ct or larger (includes Bounce/Downy Wrinkle Guard 80 ct or larger) OR In Wash Scent Boosters 8.6 oz or larger (includes Downy Unstopables, Fresh Protect, Odor Protect, and Infusions) (excludes Downy Libre Enjuague, Gain Fireworks, and trial/travel size).'

Using Regex to separate the currency/facevalue

Logic - Starts with a '$', 1 or more digits, a decimal point(\.), 0 or more digits.

In [52]:
re.findall('\$[0-9]+\.[0-9]*',text[0][0])

['$2.00']

In [88]:
FaceValue=[re.findall('\$[0-9]+\.[0-9]*',text[0][i]) for i in range(0,len(text))]

In [91]:
FaceValue

[['$2.00'],
 ['$2.00'],
 ['$2.00'],
 ['$1.00'],
 ['$3.00'],
 ['$1.11'],
 ['$1.00'],
 ['$1.00'],
 ['$0.65'],
 ['$2.00'],
 ['$0.50'],
 ['$1.00'],
 ['$0.50'],
 ['$0.50'],
 ['$1.00'],
 ['$1.00'],
 ['$1.00'],
 ['$0.50'],
 ['$2.00'],
 ['$0.50'],
 ['$1.00'],
 ['$1.25'],
 ['$2.00'],
 ['$1.00'],
 ['$1.00'],
 ['$1.00'],
 ['$1.00'],
 ['$1.00'],
 ['$1.00'],
 ['$1.00'],
 ['$1.00'],
 ['$1.00'],
 ['$0.75'],
 ['$1.00'],
 ['$3.00'],
 ['$2.00'],
 ['$4.00'],
 ['$2.00'],
 [],
 ['$1.50'],
 ['$4.00'],
 ['$3.00'],
 ['$2.00'],
 ['$4.00'],
 ['$2.00'],
 ['$2.00'],
 ['$2.00'],
 ['$5.00'],
 ['$2.00'],
 ['$3.00'],
 ['$2.00'],
 ['$2.00'],
 ['$1.50'],
 ['$1.50'],
 ['$2.00'],
 ['$3.00'],
 ['$3.00'],
 ['$2.00'],
 ['$1.00'],
 ['$8.00'],
 ['$3.00'],
 ['$0.75'],
 ['$1.50'],
 ['$3.00'],
 [],
 ['$5.00'],
 ['$3.00'],
 ['$0.75'],
 ['$3.00'],
 ['$1.00'],
 ['$5.00'],
 ['$5.00'],
 ['$4.00'],
 ['$1.00'],
 ['$4.00'],
 ['$1.00'],
 ['$2.00'],
 ['$0.75'],
 ['$4.00'],
 ['$3.00'],
 ['$2.00'],
 [],
 ['$1.00'],
 ['$0.50'],
 ['$1.50'],
 

#Removing the inner list
FaceValue=[j if j!=[] else j for i in FaceValue for j in i]

In [103]:
#To check how many entries did not return a FaceValue and their corresponding index
s=0
c=[]
for i in range(0,len(FaceValue)):
    if FaceValue[i]==[]:
        c.append(i)
        s=s+1

In [104]:
s

307

There are 307 null empty lists in FaceValue 

In [105]:
text[0][c]

38     SAVE 55¢ on any ONE (1) OFF!® Product&nbsp;(Ex...
64     SAVE 55¢ on any ONE (1) Raid® Essential Oils P...
81                  $5 1 Hot Bar Entree, 2 Sides, 1 Roll
118    SAVE 55¢ on any ONE (1) OFF!® Product(Excludes...
131    BUY ONE GET ONE FREE on any ONE (1) bag of Fla...
                             ...                        
848    SAVE 50¢ on any ONE (1) Jack Link's® Beef Jerk...
855    SAVE 55¢ on any ONE (1) OFF!® Product&nbsp;(Ex...
867    SAVE 75¢ when you buy ONE (1) bubly™ sparkling...
880    BUY TWO GET ONE Buy ONE (1) SOUR PATCH KIDS or...
884    BUY TWO GET ONE Buy ONE (1) CHIPS AHOY! Cookie...
Name: 0, Length: 307, dtype: object